<a href="https://colab.research.google.com/github/aarushkachhawa/Pinecone-Playground/blob/main/pinecone_api_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pinecone-client
from pinecone import Pinecone, ServerlessSpec
import pandas as pd

from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Pinecone-Playground

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 3.1 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/My Drive/Pinecone-Playground


## Creating Index Intro

In [ ]:
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
spec = ServerlessSpec(cloud='aws', region='us-west-2')

In [ ]:
index_name = "pinecone-intro"

if index_name in pc.list_indexes().names():
  pc.delete_index(index_name)

In [ ]:
import time

dimensions = 3
pc.create_index(
    name=index_name,
    dimension=dimensions,
    metric="cosine",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
  time.sleep(1)

In [ ]:
index = pc.Index(index_name)

In [ ]:
df = pd.DataFrame(
    data={
        "id": ["A", "B"],
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


In [ ]:
# insert vectors into the index
index.upsert(vectors=zip(df.id, df.vector))

{'upserted_count': 2}

In [ ]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
index.query(
    vector=[2., 2., 2.],
    top_k=2, # return top_k matches
    include_values=True
)

{'matches': [{'id': 'A', 'score': 1.0, 'values': [1.0, 1.0, 1.0]},
             {'id': 'B', 'score': 0.925820112, 'values': [1.0, 2.0, 3.0]}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [ ]:
pc.delete_index(index_name)